In [198]:
import os
from os import listdir
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam, RMSprop, SGD
#warnings.filterwarnings('ignore')                  # Turn the warnings off.
%matplotlib inline

In [199]:
path_init = '/Users/marcooliveira/Desktop/recaptcha'
os.chdir(path_init)

dir_true = path_init + '/filtered tractors'
dir_false = path_init + '/filtered non tractors' #escadinhas

In [200]:
print(os.getcwd())

/Users/marcooliveira/Desktop/recaptcha


In [201]:
def load_images(path):
    # return array of images

    images_list = listdir(path)
    print (listdir(path)[0])
    list_images = []
    first = True
    
    for image in images_list:
        img = Image.open(path + image)
        npframe = np.array(img.getdata())
        list_images.append (npframe.flatten())
        if first:
            first = False
            print (npframe.shape)
            print (npframe.flatten().shape)
            
    
    return pd.DataFrame(list_images)

In [202]:
imgs_true = load_images(dir_true + '/')
imgs_false = load_images(dir_false + '/')

filtered tractors2021-09-23 16-07-16.976448.png
(9801, 3)
(29403,)
filtered cars2021-09-20 23-18-54.449692.png
(9801, 3)
(29403,)


In [203]:
print(imgs_true.shape)
print(imgs_false.shape)
df = imgs_false
print (f'len = {imgs_false.shape[1]}')
if imgs_false.shape[1] == 4*99*99:
    df.drop(df.iloc[:, 3*99*99:], inplace = True, axis = 1)
    print (imgs_false)
if imgs_true.shape[1] == 4*99*99:
    imgs_true.drop(imgs_true.iloc[:, 3*99*99], inplace = True, axis = 1)
print (imgs_true.shape)
print (imgs_false.shape)

(352, 29403)
(238, 39204)
len = 39204
     0      1      2      3      4      5      6      7      8      9      \
0      192    201    200    130    136    134    178    182    181    197   
1      162    156    132    175    148    118    174    143    123    172   
2       48     56     45     58     65     47     67     72     49     71   
3      121    139    161    114    140    175    109    141    179    108   
4      153    130     89    101     78     37    164    141     99    172   
..     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
233     80     94     59     84     96     48     79     87     50     85   
234     78     63      6    113    104     49    107    105     57     82   
235    191    192    194    190    191    193    189    190    192    190   
236    119     81     78    112     83     79     92     68     66    111   
237    194    195    189    192    193    187    187    189    184    177   

     ...  29393  29394  29395  29396 

In [204]:
img_true_scaled = imgs_true/255
imgs_false_scaled = imgs_false /255

In [205]:
img_true_scaled

,0,1,2,3,4,5,6,7,8,9,...,29393,29394,29395,29396,29397,29398,29399,29400,29401,29402
0,0.847059,0.831373,0.784314,0.831373,0.811765,0.737255,0.823529,0.800000,0.713725,0.800000,...,0.372549,0.615686,0.549020,0.376471,0.596078,0.537255,0.368627,0.560784,0.509804,0.345098
1,0.270588,0.254902,0.109804,0.247059,0.235294,0.066667,0.270588,0.266667,0.086275,0.301961,...,0.266667,0.376471,0.352941,0.298039,0.388235,0.364706,0.309804,0.364706,0.333333,0.290196
2,0.301961,0.301961,0.262745,0.298039,0.298039,0.258824,0.290196,0.290196,0.250980,0.282353,...,0.560784,0.623529,0.584314,0.580392,0.584314,0.549020,0.552941,0.584314,0.568627,0.564706
3,0.733333,0.721569,0.694118,0.741176,0.725490,0.690196,0.745098,0.717647,0.678431,0.721569,...,0.145098,0.486275,0.431373,0.152941,0.494118,0.450980,0.129412,0.513725,0.486275,0.141176
4,0.329412,0.380392,0.172549,0.317647,0.427451,0.098039,0.243137,0.376471,0.047059,0.301961,...,0.133333,0.431373,0.509804,0.129412,0.431373,0.509804,0.129412,0.435294,0.513725,0.125490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,0.368627,0.333333,0.313725,0.345098,0.309804,0.290196,0.321569,0.286275,0.266667,0.321569,...,0.352941,0.313725,0.286275,0.356863,0.301961,0.294118,0.349020,0.298039,0.309804,0.337255
348,0.670588,0.647059,0.654902,0.694118,0.678431,0.674510,0.670588,0.654902,0.650980,0.658824,...,0.180392,0.341176,0.341176,0.145098,0.341176,0.337255,0.113725,0.337255,0.333333,0.113725
349,0.537255,0.556863,0.529412,0.552941,0.556863,0.486275,0.572549,0.556863,0.509804,0.560784,...,0.235294,0.384314,0.266667,0.227451,0.372549,0.254902,0.223529,0.360784,0.250980,0.207843
350,0.329412,0.298039,0.286275,0.262745,0.262745,0.215686,0.301961,0.329412,0.258824,0.274510,...,0.949020,0.921569,0.917647,0.949020,0.929412,0.925490,0.956863,0.913725,0.909804,0.941176


In [206]:
imgs_false_scaled

,0,1,2,3,4,5,6,7,8,9,...,29393,29394,29395,29396,29397,29398,29399,29400,29401,29402
0,0.752941,0.788235,0.784314,0.509804,0.533333,0.525490,0.698039,0.713725,0.709804,0.772549,...,0.219608,0.200000,0.215686,0.219608,0.200000,0.215686,0.219608,0.215686,0.215686,0.207843
1,0.635294,0.611765,0.517647,0.686275,0.580392,0.462745,0.682353,0.560784,0.482353,0.674510,...,0.486275,0.545098,0.537255,0.486275,0.541176,0.541176,0.494118,0.541176,0.541176,0.501961
2,0.188235,0.219608,0.176471,0.227451,0.254902,0.184314,0.262745,0.282353,0.192157,0.278431,...,0.168627,0.360784,0.333333,0.219608,0.443137,0.341176,0.243137,0.517647,0.368627,0.294118
3,0.474510,0.545098,0.631373,0.447059,0.549020,0.686275,0.427451,0.552941,0.701961,0.423529,...,0.494118,0.600000,0.576471,0.521569,0.596078,0.572549,0.517647,0.584314,0.560784,0.505882
4,0.600000,0.509804,0.349020,0.396078,0.305882,0.145098,0.643137,0.552941,0.388235,0.674510,...,0.639216,0.768627,0.772549,0.749020,0.756863,0.760784,0.737255,0.890196,0.894118,0.874510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,0.313725,0.368627,0.231373,0.329412,0.376471,0.188235,0.309804,0.341176,0.196078,0.333333,...,0.239216,0.333333,0.368627,0.247059,0.384314,0.419608,0.298039,0.384314,0.419608,0.298039
234,0.305882,0.247059,0.023529,0.443137,0.407843,0.192157,0.419608,0.411765,0.223529,0.321569,...,0.360784,0.686275,0.631373,0.588235,0.749020,0.694118,0.650980,0.819608,0.772549,0.725490
235,0.749020,0.752941,0.760784,0.745098,0.749020,0.756863,0.741176,0.745098,0.752941,0.745098,...,0.376471,0.478431,0.454902,0.392157,0.541176,0.513725,0.450980,0.631373,0.600000,0.549020
236,0.466667,0.317647,0.305882,0.439216,0.325490,0.309804,0.360784,0.266667,0.258824,0.435294,...,0.698039,0.674510,0.698039,0.698039,0.682353,0.698039,0.694118,0.698039,0.701961,0.682353


In [207]:
img_true_scaled['label'] = 1
imgs_false_scaled['label'] = 0

In [208]:
frames = [img_true_scaled, imgs_false_scaled]
df_final = pd.concat (frames)
df_final

,0,1,2,3,4,5,6,7,8,9,...,29394,29395,29396,29397,29398,29399,29400,29401,29402,label
0,0.847059,0.831373,0.784314,0.831373,0.811765,0.737255,0.823529,0.800000,0.713725,0.800000,...,0.615686,0.549020,0.376471,0.596078,0.537255,0.368627,0.560784,0.509804,0.345098,1
1,0.270588,0.254902,0.109804,0.247059,0.235294,0.066667,0.270588,0.266667,0.086275,0.301961,...,0.376471,0.352941,0.298039,0.388235,0.364706,0.309804,0.364706,0.333333,0.290196,1
2,0.301961,0.301961,0.262745,0.298039,0.298039,0.258824,0.290196,0.290196,0.250980,0.282353,...,0.623529,0.584314,0.580392,0.584314,0.549020,0.552941,0.584314,0.568627,0.564706,1
3,0.733333,0.721569,0.694118,0.741176,0.725490,0.690196,0.745098,0.717647,0.678431,0.721569,...,0.486275,0.431373,0.152941,0.494118,0.450980,0.129412,0.513725,0.486275,0.141176,1
4,0.329412,0.380392,0.172549,0.317647,0.427451,0.098039,0.243137,0.376471,0.047059,0.301961,...,0.431373,0.509804,0.129412,0.431373,0.509804,0.129412,0.435294,0.513725,0.125490,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,0.313725,0.368627,0.231373,0.329412,0.376471,0.188235,0.309804,0.341176,0.196078,0.333333,...,0.333333,0.368627,0.247059,0.384314,0.419608,0.298039,0.384314,0.419608,0.298039,0
234,0.305882,0.247059,0.023529,0.443137,0.407843,0.192157,0.419608,0.411765,0.223529,0.321569,...,0.686275,0.631373,0.588235,0.749020,0.694118,0.650980,0.819608,0.772549,0.725490,0
235,0.749020,0.752941,0.760784,0.745098,0.749020,0.756863,0.741176,0.745098,0.752941,0.745098,...,0.478431,0.454902,0.392157,0.541176,0.513725,0.450980,0.631373,0.600000,0.549020,0
236,0.466667,0.317647,0.305882,0.439216,0.325490,0.309804,0.360784,0.266667,0.258824,0.435294,...,0.674510,0.698039,0.698039,0.682353,0.698039,0.694118,0.698039,0.701961,0.682353,0


In [209]:
#df_final = shuffle(df_final)
#df_final
shuffled = df_final
shuffled = shuffled.sample (frac = 1)

In [210]:
labels = shuffled['label']
df_shuffled = shuffled.drop(columns = 'label')

In [211]:
X_train, X_test, y_train, y_test = train_test_split(df_shuffled, labels, test_size = 0.3, random_state = 123)

In [212]:
X_test.shape

(177, 29403)

In [213]:
X_train = X_train.to_numpy().reshape(-1, 99, 99, 3)
X_test = X_test.to_numpy().reshape(-1, 99, 99, 3)

In [214]:
drop_prob = 0.5

my_model = Sequential()
# 1st convolution + pooling.
my_model.add(Conv2D(input_shape=(99,99,3),filters=8,kernel_size=(5,5),padding='same',activation="relu"))        # 1 channel of grayscale.
my_model.add(MaxPooling2D(pool_size=2))
# 2nd convolution + pooling.
my_model.add(Conv2D(filters=16,kernel_size=(5,5), padding='same',activation="relu"))
my_model.add(MaxPooling2D(pool_size=2))
#3rd
my_model.add(Conv2D(filters=32,kernel_size=(5,5), padding='same',activation="relu"))
my_model.add(MaxPooling2D(pool_size=2))
# Flattened fully connected layer.
my_model.add(Flatten())
my_model.add(Dense(units = 1024, activation="relu"))
# Apply dropout.
my_model.add(Dropout(rate=drop_prob))
# Output layer.
my_model.add(Dense(units = 1, activation="sigmoid"))

In [215]:
my_model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 99, 99, 8)         608       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 49, 49, 8)         0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 49, 49, 16)        3216      
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 24, 24, 32)        12832     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 4608)             

In [216]:
# Hyperparameters.
n_epochs = 35
batch_size = 200
learn_rate = 0.0005

In [217]:
my_optimizer=Adam(learning_rate=learn_rate)
my_model.compile(loss = "binary_crossentropy", optimizer = my_optimizer, metrics=["accuracy"])

In [218]:
my_summary = my_model.fit(X_train, y_train, epochs=n_epochs, batch_size = batch_size, validation_split = 0.2, verbose = 1)

Epoch 1/35
2/2 [==============================] - 2s 715ms/step - loss: 0.6918 - accuracy: 0.5019 - val_loss: 0.7233 - val_accuracy: 0.4819
Epoch 2/35
2/2 [==============================] - 1s 396ms/step - loss: 0.6555 - accuracy: 0.6675 - val_loss: 0.6466 - val_accuracy: 0.6747
Epoch 3/35
2/2 [==============================] - 1s 370ms/step - loss: 0.6265 - accuracy: 0.7536 - val_loss: 0.7512 - val_accuracy: 0.4819
Epoch 4/35
2/2 [==============================] - 1s 402ms/step - loss: 0.6123 - accuracy: 0.6316 - val_loss: 0.6038 - val_accuracy: 0.6867
Epoch 5/35
2/2 [==============================] - 1s 391ms/step - loss: 0.5528 - accuracy: 0.7386 - val_loss: 0.5589 - val_accuracy: 0.7229
Epoch 6/35
2/2 [==============================] - 1s 412ms/step - loss: 0.5128 - accuracy: 0.7503 - val_loss: 0.6042 - val_accuracy: 0.6506
Epoch 7/35
2/2 [==============================] - 1s 463ms/step - loss: 0.4882 - accuracy: 0.7420 - val_loss: 0.4995 - val_accuracy: 0.7952
Epoch 8/35
2/2 [====

In [219]:
ACC = my_model.evaluate(X_test, y_test, verbose=0)[1]    
print("Test Accuracy : {}".format(np.round(ACC,3)))

Test Accuracy : 0.825
